In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('D:/farzad/Artificial Intelligence/Sem2/NLP/Consumer_Complaints.csv')

In [3]:
data.dropna(inplace=True)
X = data['Consumer complaint narrative']

y = data['Product']

In [10]:
y.isnull().sum()

0

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [12]:
tfidf = TfidfVectorizer(stop_words='english')
vector = tfidf.fit_transform(data['Consumer complaint narrative'])

In [13]:
le = LabelEncoder()
y = le.fit_transform(data['Product'])

In [14]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=5,algorithm='randomized',n_iter=100,random_state=42)

In [15]:
svd.fit(vector)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=100,
       random_state=42, tol=0.0)

In [16]:
len(svd.components_)

5

In [17]:
terms = tfidf.get_feature_names()
topics = []

In [21]:
for i, comp in enumerate(svd.components_):
    #print(i)
    #print(comp)
    terms_comp = zip(terms,comp)
    sorted_terms = sorted(terms_comp, key=lambda x: x[1],reverse=True)[:7]
    #print(sorted_terms)
    topics.append("topic " +str(i)+": ")
    for t in sorted_terms:
        topics.append(t[0])
final_topic_list = [topics[i:i+8] for i in range(0, len(topics),8)]
for x in final_topic_list:
    print(x)

['topic 0: ', 'xxxx', 'xx', 'debt', 'credit', 'account', 'collection', '00']
['topic 1: ', 'xx', 'debt', 'credit', 'report', 'account', 'company', 'collection']
['topic 2: ', 'xx', 'xxxx', '00', '2016', 'loans', '2017', 'dated']
['topic 3: ', 'calls', 'phone', 'number', 'calling', 'day', 'times', 'xx']
['topic 4: ', '00', 'payment', 'pay', 'loan', 'paid', 'payments', 'told']
['topic 0: ', 'xxxx', 'xx', 'debt', 'credit', 'account', 'collection', '00']
['topic 1: ', 'xx', 'debt', 'credit', 'report', 'account', 'company', 'collection']
['topic 2: ', 'xx', 'xxxx', '00', '2016', 'loans', '2017', 'dated']
['topic 3: ', 'calls', 'phone', 'number', 'calling', 'day', 'times', 'xx']
['topic 4: ', '00', 'payment', 'pay', 'loan', 'paid', 'payments', 'told']
['topic 0: ', 'xxxx', 'xx', 'debt', 'credit', 'account', 'collection', '00']
['topic 1: ', 'xx', 'debt', 'credit', 'report', 'account', 'company', 'collection']
['topic 2: ', 'xx', 'xxxx', '00', '2016', 'loans', '2017', 'dated']
['topic 3: ', '

In [22]:
import re
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import logging
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [23]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from collections import Counter
import operator

In [36]:
text = list(X.values)

text = text[:1000]
text = [str(x).lower() for x in text]

stop = list(stopwords.words('english'))
exclude = set(punctuation)
lemma = WordNetLemmatizer()
stop=stop+['x','xx','xxx','xxxx','xxxxxxxx',"x","xx","xxx","xxxx","xxxxxxxx"]
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [37]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized  = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in text]
print(doc_clean[:2])

[['called', 'cell', 'phone', 'first', 'national', 'debt', 'inc', 'told', 'owed', '21000', 'debt', 'incurred', 'xxxxxxxx', 'xxxx', 'debt', 'never', 'resolved', 'time', 'returned', 'home', 'found', 'debt', 'longer', 'bank', 'moved', 'something', 'website', 'number', 'recording', 'find', 'bought', 'another', 'bank', 'debt', 'came', 'xxxxxxxx', 'xxxxxxxx', 'xxxxxxxx', 'xxxxxxxx', 'debt', 'nine', 'year', 'old'], ['sent', 'cease', 'desist', 'letter', 'medical', 'debt', 'collection', 'agency', 'bill', 'recently', 'deceased', 'husband', 'ca', 'nt', 'afford', 'pay', 'certified', 'mail', 'receipt', 'payment', 'made', 'never', 'got', 'receipt', 'back', 'get', '2', 'robo', 'collection', 'call', 'day', '4', 'day', 'week', 'want', 'call', 'stop']]


In [38]:
import gensim
from gensim import corpora

In [40]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


In [45]:
#print([[(dictionary[id]) for ]])
print ([[(dictionary[id], freq) for id, freq in cp] for cp in doc_term_matrix[:1]])

[[('21000', 1), ('another', 1), ('bank', 2), ('bought', 1), ('called', 1), ('came', 1), ('cell', 1), ('debt', 6), ('find', 1), ('first', 1), ('found', 1), ('home', 1), ('inc', 1), ('incurred', 1), ('longer', 1), ('moved', 1), ('national', 1), ('never', 1), ('nine', 1), ('number', 1), ('old', 1), ('owed', 1), ('phone', 1), ('recording', 1), ('resolved', 1), ('returned', 1), ('something', 1), ('time', 1), ('told', 1), ('website', 1), ('xxxx', 1), ('xxxxxxxx', 5), ('year', 1)]]


In [46]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(corpus = doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=5, num_words=7))

[(0, '0.005*"s" + 0.005*"sent" + 0.004*"credence" + 0.004*"xxxx2017" + 0.004*"paper" + 0.004*"xxxx" + 0.004*"wf"'), (1, '0.024*"loan" + 0.013*"would" + 0.013*"xxxx" + 0.011*"debt" + 0.010*"company" + 0.009*"called" + 0.008*"pay"'), (2, '0.032*"xxxx" + 0.014*"debt" + 0.012*"payment" + 0.010*"loan" + 0.008*"bank" + 0.008*"account" + 0.007*"year"'), (3, '0.024*"call" + 0.018*"xxxx" + 0.012*"phone" + 0.011*"time" + 0.011*"bill" + 0.011*"number" + 0.010*"called"'), (4, '0.032*"debt" + 0.030*"credit" + 0.024*"xxxx" + 0.020*"collection" + 0.017*"account" + 0.015*"report" + 0.011*"letter"')]


In [34]:
!pip install pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\Farzad Avari\AppData\Local\conda\conda\envs\ImageEnv1\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      48.542126        1       1  0.110319 -0.021921
0      27.126247        1       2  0.064554 -0.053904
2      18.897959        1       3  0.116847  0.092152
3       3.921080        1       4 -0.107831 -0.103500
1       1.512588        1       5 -0.183889  0.087173, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
5742  Default  1314.000000          debt  1314.000000  30.0000  30.0000
3908  Default  8118.000000          xxxx  8118.000000  29.0000  29.0000
4121  Default   613.000000          call   613.000000  28.0000  28.0000
4567  Default   405.000000        number   405.000000  27.0000  27.0000
973   Default   558.000000       payment   558.000000  26.0000  26.0000
4936  Default   398.000000          loan   398.000000  25.0000  25.0000
3301  Default   347.000000   information   347.000000  24.0000  24.0000
46    Default   364.000000         phone   364.000000  23.0000  23.0000
3465  Default   551.000000       company   551.000000  22.0000  22.0000
1031  Default   834.000000       account   834.000000  21.0000  21.0000
5338  Default   277.000000             s   277.000000  20.0000  20.0000
5831  Default   177.000000       calling   177.000000  19.0000  19.0000
4840  Default  1108.000000        credit  1108.000000  18.0000  18.0000
4294  Default   780.000000    collection   780.000000  17.0000  17.0000
5594  Default   463.000000          time   463.000000  16.0000  16.0000
2286  Default   401.000000        called   401.000000  15.0000  15.0000
1091  Default   266.000000          also   266.000000  14.0000  14.0000
4290  Default   353.000000          sent   353.000000  13.0000  13.0000
5528  Default   167.000000        charge   167.000000  12.0000  12.0000
3731  Default   105.000000        person   105.000000  11.0000  11.0000
1129  Default   131.000000         check   131.000000  10.0000  10.0000
885   Default   158.000000          date   158.000000   9.0000   9.0000
561   Default    94.000000       provide    94.000000   8.0000   8.0000
483   Default   307.000000           day   307.000000   7.0000   7.0000
2427  Default    85.000000       federal    85.000000   6.0000   6.0000
3828  Default    46.000000           erc    46.000000   5.0000   5.0000
2770  Default   100.000000       without   100.000000   4.0000   4.0000
2162  Default   347.000000       service   347.000000   3.0000   3.0000
112   Default   347.000000      xxxxxxxx   347.000000   2.0000   2.0000
1370  Default   209.000000          bank   209.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
57     Topic5     2.228155      timeline     5.849065   3.2262  -6.3155
741    Topic5     3.494301             b    14.408685   2.7747  -5.8655
5184   Topic5     2.223610  fraudulently     6.774335   3.0773  -6.3175
3713   Topic5     4.738362    limitation    32.094234   2.2784  -5.5609
538    Topic5     6.321050      personal    59.382305   1.9512  -5.2727
4567   Topic5    15.250974        number   405.715118   0.9103  -4.3920
5047   Topic5     2.554570             c     9.488029   2.8792  -6.1787
118    Topic5     3.976374    individual    29.210220   2.1972  -5.7363
5742   Topic5    19.681532          debt  1314.221924  -0.0100  -4.1369
561    Topic5     5.954582       provide    94.917885   1.4225  -5.3325
3446   Topic5     4.852595           act    58.391224   1.7037  -5.5371
4764   Topic5     2.100600          harm     7.827760   2.8759  -6.3744
5338   Topic5     7.976988             s   277.009857   0.6439  -5.0401
3002   Topic5     2.028161    personally     7.913247   2.8299  -6.4095
3301   Topic5     7.703657   information   347.861511   0.3812  -5.0749
3731   Topic5     5.058847        person   105.849663   1.1505  -5.4955
4519   Topic5     2.281471     allegedly    13.3973